In [1]:
import pandas as pd
import torch
import torch.nn as nn
from torch.optim import Adam
from torchvision import models
from pathlib import Path
import json
from tqdm.notebook import tqdm

# Local modules
import sys
sys.path.append('..')
from src.model.dataset import create_data_loaders
from src.model.transforms import get_train_transforms,get_val_transforms

# Training settings
IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 3
LEARNING_RATE = 3e-4
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# Get reproducible results by controlling random
torch.manual_seed(42)
if DEVICE == 'cuda':
    torch.cuda.manual_seed(42)

print(f'Selected device: {DEVICE}')


Selected device: cuda


In [2]:
# Data and DataLoader
data_path = "../../data/processed/01_final_dataset"

train_loader,val_loader,test_loader,weights = create_data_loaders(
    data_path,
    get_train_transforms(IMG_SIZE),
    get_val_transforms(IMG_SIZE),
    BATCH_SIZE,
    class_weights=True
)

# Class names
with open('../../data/processed/01_metadata/class_names.json','r') as f:
    class_to_idx = json.load(f)
    NUM_CLASSES = len(class_to_idx)

print(f"Total class number: {NUM_CLASSES}")
print(f"In train set: {len(train_loader.dataset)} images")
print(f"In val set: {len(val_loader.dataset)} images")
print(f"In test set: {len(test_loader.dataset)} images")

102951 images loaded from dataset (train).
22036 images loaded from dataset (val).
22129 images loaded from dataset (test).
Total class number: 197
In train set: 102951 images
In val set: 22036 images
In test set: 22129 images


In [3]:
def create_model(num_classes, model_name='mobilenet_v2',fine_tune=False):
    """
        Download pretrained model and update last layer or fine-tuning
        Args:
        num_classes (int): Class number on dataset
        model_name (str): name of the selected model
        fine_tune (bool): Determines fine-tuning or last layer updating

    Returns:
        torch.nn.Module: Customized model object
    """
    if model_name == 'mobilenet_v2':
        model = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.IMAGENET1K_V1)
        num_features = model.classifier[-1].in_features
        model.classifier[-1] = nn.Linear(num_features, num_classes)
    elif model_name == 'mobilenet_v3':
        model = models.mobilenet_v3_small(weights=models.MobileNet_V3_Small_Weights.IMAGENET1K_V1)
        num_features = model.classifier[-1].in_features
        model.classifier[-1] = nn.Linear(num_features, num_classes)
    elif model_name == 'resnet18':
        model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
        num_features = model.fc.in_features
        model.fc = nn.Linear(num_features, num_classes)
    else:
        print(f"Error: '{model_name}' does not support. Please select 'mobilenet_v2', 'mobilenet_v3' or 'resnet18'.")

    # Fine-tuning
    for param in model.parameters():
        param.requires_grad = fine_tune

    return model

model_name = 'mobilenet_v3'
model = create_model(NUM_CLASSES,model_name,fine_tune=False)
model.to(DEVICE)

print(f'{model_name} was successfully created.')
print(model)

mobilenet_v3 was successfully created.
MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): SqueezeExcitation(
          (avgpool): AdaptiveAvgPool2d(output_size=1)
          (fc1): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
          (fc2): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1))
          (activation): ReLU()
          (scale_activation): Hardsigmoid()
        )
        (2): Conv2dNormActivation(
          (

In [4]:
# Loss and Optimizer
criterion = nn.CrossEntropyLoss(weight=weights.to(DEVICE) if weights is not None else None)
optimizer = Adam(model.parameters(),lr=LEARNING_RATE)

def train_model(model,train_loader,val_loader,criterion,optimizer,epochs,device,model_name):
    best_val_loss = float('inf')
    best_val_acc = 0.0

    history = []

    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        train_corrects = 0

        for images,labels in tqdm(train_loader,desc=f"Epoch {epoch+1}/{epochs} (Train)"):
            images,labels = images.to(DEVICE),labels.to(DEVICE)

            optimizer.zero_grad()

            outputs = model(images)
            loss = criterion(outputs,labels)

            loss.backward()
            optimizer.step()

            train_loss += loss.item() * images.size(0)
            _,predicts = torch.max(outputs,1)
            train_corrects += torch.sum(predicts == labels.data)
        train_loss /= len(train_loader.dataset)
        train_acc = train_corrects.double() / len(train_loader.dataset)

        # Validation
        model.eval()
        val_loss = 0.0
        val_corrects = 0

        with torch.no_grad():
            for images, labels in tqdm(val_loader, desc=f"Epoch {epoch+1}/{epochs} (Validation)"):
                images, labels = images.to(device), labels.to(device)

                outputs = model(images)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * images.size(0)

                _, predictions = torch.max(outputs, 1)
                val_corrects += torch.sum(predictions == labels.data)

        val_loss /= len(val_loader.dataset)
        val_acc = val_corrects.double() / len(val_loader.dataset)

        print(f"Epoch {epoch+1}/{epochs} -> Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

        history.append({
            'epoch': epoch + 1,
            'train_loss': train_loss,
            'train_acc': train_acc.item(),
            'val_loss': val_loss,
            'val_acc': val_acc.item()
        })

        # Save
        if val_acc > best_val_acc:
            best_val_acc = val_acc

            # Model path
            model_path = Path(f"../../models/{model_name}_best_model.pt")
            torch.save(model.state_dict(), model_path)
            print(f"Model was saved: {model_name}")

    history_df = pd.DataFrame(history)

    # Save as csv
    history_path = Path(f"../../outputs/logs/{model_name}_training_history.csv")
    history_df.to_csv(history_path,index=False)
    print(f"Training metrics was saved: {model_name}_training_history.csv")

# Start training
train_model(
    model,
    train_loader,
    val_loader,
    criterion,
    optimizer,
    epochs = EPOCHS,
    device = DEVICE,
    model_name=model_name
)

print("\n Finished")


Epoch 1/3 (Train):   0%|          | 0/1609 [00:00<?, ?it/s]

FileNotFoundError: Caught FileNotFoundError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "C:\Users\User\.conda\envs\diet_assistant_env\Lib\site-packages\torch\utils\data\_utils\worker.py", line 349, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
           ^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\User\.conda\envs\diet_assistant_env\Lib\site-packages\torch\utils\data\_utils\fetch.py", line 52, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
            ~~~~~~~~~~~~^^^^^
  File "C:\Users\User\Desktop\diet-assistant\src\model\dataset.py", line 53, in __getitem__
    image = Image.open(img_path).convert('RGB')
            ^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\User\.conda\envs\diet_assistant_env\Lib\site-packages\PIL\Image.py", line 3469, in open
    fp = builtins.open(filename, "rb")
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\User